### Now, we want to go over an example of wrapping an existing Python software package
Software package we'll be working with: https://github.com/snap-stanford/snapvx. 
Other software packages that need to be installed:
* http://snap.stanford.edu/snappy/
* http://www.cvxpy.org/en/latest/

Make sure to follow installation instructions from all these links, and once all have been successfully installed move on to the next step

#### First we want to make sure the Python scripts work. We'll use three simple examples:
link to the [example Python codes](Python_codes.ipynb) from snapvx

### Example 1

In [4]:
using PyCall #make sure to have the package added already via Pkg.add("PyCall")

In [5]:
@pyimport snapvx

In [6]:
gvx = snapvx.TGraphVX()

PyObject <snapvx.TGraphVX; proxy of <Swig Object of type 'TUNGraph *' at 0x31b00c5d0> >

In [7]:
@pyimport cvxpy

In [8]:
x1 = cvxpy.Variable(1, name="x1")

PyObject Variable(1, 1)

In [9]:
obj = cvxpy.square(x1)

PyObject Expression(CONVEX, POSITIVE, (1, 1))

In [10]:
@pyimport cvxpy.constraints as cvxpyconstraints

In [11]:
gvxconstraints = cvxpyconstraints.LeqConstraint(x1,10)

PyObject LeqConstraint(Variable(1, 1), 10)

In [12]:
gvx[:AddNode](1, Objective=obj, Constraints=[gvxconstraints])

1

In [14]:
x2 = cvxpy.Variable(1, name="x2")

PyObject Variable(1, 1)

needed to add this line (`from cvxpy.expressions.expression import Expression`) to `/Users/hudanassar/anaconda/lib/python2.7/site-packages/cvxpy/expressions/__init__.py` for the following to work

In [16]:
@pyimport cvxpy.expressions as cvxpyexpressions

In [17]:
tt = cvxpyexpressions.Expression[:__add__](x2,3)

PyObject Expression(AFFINE, UNKNOWN, (1, 1))

In [18]:
obj2 = cvxpy.abs(tt)

PyObject Expression(CONVEX, POSITIVE, (1, 1))

In [19]:
obj2 = cvxpy.abs(tt)

PyObject Expression(CONVEX, POSITIVE, (1, 1))

In [24]:
    gvx[:AddNode](2, Objective=obj2, Constraints=[])

2

In [25]:
tt2 = cvxpyexpressions.Expression[:__sub__](x1,x2)

PyObject Expression(AFFINE, UNKNOWN, (1, 1))

In [26]:
gvx[:AddEdge](1, 2, Objective=cvxpy.square(cvxpy.norm(tt2)), Constraints=[])

-1

In [28]:
gvx[:Solve]()

In [30]:
gvx[:PrintSolution]()

Status: Optimal
Total Objective: 2.500335
Node 1:
  x1 [-0.51706729]
Node 2:
  x2 [-1.02366536]


### Example 2

In [1]:
# most of these have already been added - but for the sake of making examples independent from each other
using PyCall
@pyimport cvxpy.expressions as cvxpyexpressions
@pyimport numpy
@pyimport cvxpy
@pyimport snapvx

In [2]:
numpy.random[:seed](1)
num_edges = 30
num_nodes = 10
n = 10
snapGraph = snapvx.GenRndGnm(snapvx.PUNGraph,num_nodes,num_edges)

PyObject <snap.PUNGraph; proxy of <Swig Object of type 'PUNGraph *' at 0x315950b10> >

In [3]:
function laplace_reg(src,dst,data)
    e1 = src["x"]
    e2 = dst["x"]
    ex = cvxpyexpressions.Expression[:__sub__](e1,e2)
    return(snapvx.sum_squares(ex),[])
end

laplace_reg (generic function with 1 method)

In [4]:
gvx = snapvx.TGraphVX(snapGraph)

PyObject <snapvx.TGraphVX; proxy of <Swig Object of type 'TUNGraph *' at 0x3159505d0> >

In [5]:
for i = 0:num_nodes-1
    x = cvxpy.Variable(n, name="x")
    a = numpy.random[:randn](n)
    ex = cvxpyexpressions.Expression[:__sub__](x,a)
    gvx[:SetNodeObjective](i,snapvx.square(cvxpy.norm(ex)))
end

In [6]:
gvx[:AddEdgeObjectives](laplace_reg)

In [7]:
gvx[:Solve]()

In [8]:
gvx[:PrintSolution]()

Status: Optimal
Total Objective: 57.155315
Node 0:
  x [ 0.29378266 -0.26524219 -0.25250324 -0.23300168  0.41298769 -0.86914119  0.40773391  0.08331969  0.37224637  0.30800445]
Node 1:
  x [ 0.28712282 -0.61276469 -0.22615739 -0.08915738  0.46016299 -0.65311469 -0.02752935  0.08996956  0.30531097  0.49308243]
Node 2:
  x [-0.16487666  0.04105045 -0.01458014  0.04529752  0.42459157 -0.60412203  0.13055647  0.13764805  0.29633032  0.39071433]
Node 3:
  x [-0.03418349 -0.27205432 -0.26004528 -0.14080719  0.21878818 -0.51484094 -0.02752153  0.30287523  0.54348138  0.44594498]
Node 4:
  x [-0.0570535  -0.29944952 -0.27701554  0.28553805  0.2666139  -0.53738026  0.11731259  0.68169612  0.35534438  0.44016973]
Node 5:
  x [ 0.01863543 -0.20670528 -0.28652516 -0.03612443  0.26639766 -0.45263951  0.19732264  0.41410293  0.38650289  0.45014487]
Node 6:
  x [-0.18936785  0.04308614 -0.07898234 -0.00796519  0.32889441 -0.47519744  0.26286999  0.57228709  0.68563219  0.12939756]
Node 7:
  x [-0.156

In [9]:
gvx[:Solve](UseADMM=false)

In [11]:
gvx[:PrintSolution]()

Status: optimal
Total Objective: 57.135012
Node 0:
  x [ 0.29625344 -0.27553627 -0.26165741 -0.23564755  0.42884901 -0.89699685  0.41481629  0.09585558  0.39029586  0.32635133]
Node 1:
  x [ 0.29014128 -0.62653188 -0.23544224 -0.09053003  0.47580373 -0.67995861 -0.02468094  0.10288205  0.32208228  0.51312298]
Node 2:
  x [-0.16510383  0.03333938 -0.02225088  0.0445439   0.44048596 -0.6309419   0.13775923  0.151717    0.31419608  0.40774836]
Node 3:
  x [-0.03207381 -0.28351252 -0.26897479 -0.1426932   0.23373868 -0.54096335 -0.02275894  0.31593471  0.5610455   0.46414995]
Node 4:
  x [-0.05718563 -0.30948962 -0.28578127  0.2870832   0.28122733 -0.562517    0.12268358  0.69912136  0.37269646  0.45815137]
Node 5:
  x [ 0.01908112 -0.21567077 -0.29487512 -0.03661693  0.28119362 -0.47827873  0.20367474  0.42881449  0.40398699  0.46732561]
Node 6:
  x [-0.19091606  0.03527435 -0.08681512 -0.00756938  0.34392577 -0.50092702  0.26970618  0.58910844  0.7048484   0.14575171]
Node 7:
  x [-0.155

In [12]:
node1val = gvx[:GetNodeValue](1,"x")
println("Solution at Node 1 is $node1val")

Solution at Node 1 is [0.2901412758162775,-0.626531884648983,-0.2354422361912734,-0.09053002686417683,0.47580372677031607,-0.6799586099055238,-0.024680943747762363,0.10288204891173991,0.32208228390099153,0.5131229786262361]


### notice some subtle changes from the python code For example:
`gvx.solve(UseADMM=False)` becomes `gvx[:Solve](UseADMM=false)`

`x = Variable(n,name='x')` becomes `x = cvxpy.Variable(n, name="x")`

### Example 3

In [14]:
# most of these have already been added - but for the sake of making examples independent from each other
using PyCall
@pyimport cvxpy.expressions as cvxpyexpressions
@pyimport numpy
@pyimport cvxpy
@pyimport snapvx

In [15]:
function node_obj(data)
    x = cvxpy.Variable(1,name="x")
    f = float(data[1])
    ex = cvxpyexpressions.Expression[:__sub__](x,f)
    return snapvx.norm(ex)
end

node_obj (generic function with 1 method)

In [17]:
function laplace_reg(src,dst,data)
    e1 = src["x"]
    e2 = dst["x"]
    ex = cvxpyexpressions.Expression[:__sub__](e1,e2)
    return(snapvx.sum_squares(ex))
end

laplace_reg (generic function with 1 method)

In [18]:
gvx = snapvx.LoadEdgeList("BulkLoadEdges.edges")

PyObject <snapvx.TGraphVX; proxy of <Swig Object of type 'TUNGraph *' at 0x31f8c3300> >

In [19]:
gvx[:AddNodeObjectives]("BulkLoadData.csv", node_obj)

In [20]:
gvx[:AddEdgeObjectives](laplace_reg)

In [21]:
gvx[:Solve]()

In [22]:
gvx[:PrintSolution]()

Status: Optimal
Total Objective: 2.750145
Node 1:
  x [ 0.34398022]
Node 2:
  x [ 0.85601343]


## Making the wrapped python code somewhat more friendly
In these three examples, we saw some lines of code becoming a little ugly to read. For example, the line `ex = cvxpyexpressions.Expression[:__sub__](e1,e2)` is simple an `e1-e2` but since subtraction is not defined on cvxpy expressions we had to write the above line. 

This doesn't need to be the case. Let's look at the following example

In [40]:
var1 = cvxpy.Variable(1, name="var1") #our first variable
var2 = cvxpy.Variable(1, name="var2") #our first variable

PyObject Variable(1, 1)

In [41]:
exp1 = cvxpyexpressions.Expression[:__sub__](var1,var2) #our first expression
typeof(exp1)

PyCall.PyObject

In [31]:
@show typeof(var1)
var1 - var2

typeof(var1) = PyCall.PyObject


LoadError: LoadError: MethodError: `-` has no method matching -(::PyCall.PyObject, ::PyCall.PyObject)
while loading In[31], in expression starting on line 2

In [57]:
import Base.-
function -(var1::PyCall.PyObject,var2::PyCall.PyObject)
    cvxpyexpressions.Expression[:__sub__](var1,var2)
end

- (generic function with 205 methods)

In [58]:
var1-var2

PyObject Expression(AFFINE, UNKNOWN, (1, 1))

### Let's add more functions and redo the first example
Link to example1 [new code](example1_redo.ipynb)